In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split
import lightgbm as lgbm

In [6]:
df = pd.read_csv('data\\tmp_df.csv')
df.head()

,date,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.9205,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.9967,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 00:02:00,6.39107,-2.61173,-13.3271,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 00:03:00,6.44897,-2.61525,-13.3299,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 00:04:00,6.58758,-2.73082,-13.2361,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
target = pd.read_csv('data\\kpIndex.csv', sep=';', header=None)
target.columns = ['date', 'kp']
target['kp_test'] = target['kp'].shift(-1)
result_train = df.merge(target, on='date', how='left')

In [8]:
train_data = np.array([])
train_labels = np.array([])

for i in range(0, 3277440, 180):
    tmp_df = result_train.iloc[i : i+180]
    label = tmp_df.iloc[0]['kp_test']
    data = tmp_df.drop(columns=['date', 'kp', 'kp_test']).mean()
    data = np.array(data)
    train_data = np.append([train_data], data)
    train_labels = np.append([train_labels], label)

In [9]:
target.head(10)

,date,kp,kp_test
0,2016-01-01 00:00:00,6.0,5.3
1,2016-01-01 03:00:00,5.3,5.0
2,2016-01-01 06:00:00,5.0,3.3
3,2016-01-01 09:00:00,3.3,1.7
4,2016-01-01 12:00:00,1.7,1.0
5,2016-01-01 15:00:00,1.0,2.0
6,2016-01-01 18:00:00,2.0,1.7
7,2016-01-01 21:00:00,1.7,3.0
8,2016-01-02 00:00:00,3.0,2.3
9,2016-01-02 03:00:00,2.3,3.0


In [11]:
train_data = np.array([])
train_labels = np.array([])

for i in range(0, 3277440, 180):
    tmp_df = result_train.iloc[i : i+180]
    label = tmp_df.iloc[0]['kp_test']
    data = tmp_df.drop(columns=['date', 'kp', 'kp_test']).mean()
    data = np.array(data)
    train_data = np.append([train_data], data)
    train_labels = np.append([train_labels], label)

In [13]:
X_train = train_data.reshape(18208, 53)
y_train = train_labels.reshape(18208, 1)
y_train = np.nan_to_num(y_train)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [15]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae_score = mean_squared_error(y_pred, y_test)
print(mae_score)

0.5903187061323374


In [16]:
result_train.to_csv('aaaaaa.csv', index=False)